<a href="https://colab.research.google.com/github/sergiocostaifes/PPCOMP_DM/blob/main/notebooks/04_window_5min_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Título: 04_window_5min_base.ipynb
Objetivo: criar janelas de 5 minutos (minute_bucket) e gerar dataset base agregado para análises e feature engineering.
Entrada: 02-processed/google_trace_clean.parquet
Saídas:

03-features/window_5min_base.parquet

03-features/window_5min_series.parquet

In [ ]:
# =========================
# 04_window_5min_base.ipynb
# Janelamento 5 minutos + agregações base
# =========================

import pandas as pd
import numpy as np

CLEAN_PARQUET = PROCESSED_PATH / "google_trace_clean.parquet"
assert CLEAN_PARQUET.exists(), f"Não achei: {CLEAN_PARQUET}"

df = pd.read_parquet(CLEAN_PARQUET)
log(f"Carregado: {df.shape}")

# Checagens mínimas
required_cols = {"time", "t_rel_us", "hour", "failed"}
missing = required_cols - set(df.columns)
assert not missing, f"Colunas faltando no dataset limpo: {missing}"

df[["time","t_rel_us","hour","failed"]].head()

Construir minute_bucket (5 min)

Definição:

5 min = 300s = 300.000.000 µs

In [ ]:
# 5 minutos em microssegundos
WINDOW_US = 300_000_000  # 300s

df["minute_bucket"] = (df["t_rel_us"] // WINDOW_US).astype("int64")

log(f"minute_bucket min..max: {df['minute_bucket'].min()}..{df['minute_bucket'].max()}")
log(f"Número de buckets distintos: {df['minute_bucket'].nunique()}")

Agregações base por janela

Aqui vamos fazer agregações “universais”, que não dependem de colunas opcionais.

events_total: nº de eventos no bucket

failures_total: nº de falhas (failed==1) no bucket

fail_rate: failures_total / events_total

Se existirem colunas como event, machine_id, job_id etc., adicionamos agregações extras sem quebrar.

In [ ]:
group = df.groupby("minute_bucket")

window_base = pd.DataFrame({
    "events_total": group.size(),
    "failures_total": group["failed"].sum(),
})

window_base["fail_rate"] = window_base["failures_total"] / window_base["events_total"]

# agregações opcionais (se existirem no dataset)
if "event" in df.columns:
    # contagem de eventos FAIL (redundante com failed, mas útil para auditoria)
    window_base["fail_event_count"] = group["event"].apply(lambda s: (s == "FAIL").sum())

if "machine_id" in df.columns:
    window_base["unique_machines"] = group["machine_id"].nunique()

if "job_id" in df.columns:
    window_base["unique_jobs"] = group["job_id"].nunique()

# reset index para virar tabela
window_base = window_base.reset_index()

log(f"window_base: {window_base.shape}")
window_base.head()

Criar série completa contínua (reindex)

Isso é importante para:

análise temporal

detectar episódios depois (sem buracos)

padronizar dataset para treino

In [ ]:
mb_min = int(window_base["minute_bucket"].min())
mb_max = int(window_base["minute_bucket"].max())

full_index = pd.DataFrame({"minute_bucket": np.arange(mb_min, mb_max + 1, dtype="int64")})

window_series = full_index.merge(window_base, on="minute_bucket", how="left").fillna(0)

# garantir tipos numéricos corretos
for col in window_series.columns:
    if col != "minute_bucket":
        window_series[col] = pd.to_numeric(window_series[col], errors="coerce").fillna(0)

log(f"window_series (completo): {window_series.shape}")
window_series.head()

Salvar Parquet

In [ ]:
OUT_BASE   = FEATURES_PATH / "window_5min_base.parquet"
OUT_SERIES = FEATURES_PATH / "window_5min_series.parquet"

window_base.to_parquet(OUT_BASE, index=False, compression="snappy")
window_series.to_parquet(OUT_SERIES, index=False, compression="snappy")

log(f"Salvo: {OUT_BASE}")
log(f"Salvo: {OUT_SERIES}")

Sanidade final rápida

In [ ]:
log("Sanidade rápida")
log(f"Total events_total (base): {int(window_base['events_total'].sum())}")
log(f"Total failures_total (base): {int(window_base['failures_total'].sum())}")

top = window_series.sort_values("failures_total", ascending=False).head(10)
top